# Titanic

https://www.kaggle.com/c/titanic/overview

In [283]:
import pandas as pd
import numpy as np

data = pd.read_csv("train.csv")
data.fillna(data.mean(),axis=0,inplace=True)
x = data.drop(["Name","Ticket","PassengerId","Survived","Embarked"],axis=1)
y = data["Survived"]
#x["Embarked"] = x["Embarked"].apply(lambda x: "nan" if pd.isnull(x) else x)
x["Cabin"] = x["Cabin"].apply(lambda x: 0 if pd.isnull(x) else 1)

Detalles
* Funciona mejor ignorando la columna Embarked pero no hay mucha diferencia
* El árbol funciona relativamente bien con 10 elementos mínimos por hoja

## Holdout

In [284]:
from sklearn.preprocessing import LabelEncoder

le_sex = LabelEncoder()
le_sex.fit(x["Sex"])
x["Sex"] = le_sex.transform(x["Sex"])

#le_embarked = LabelEncoder()
#le_embarked.fit(x["Embarked"])
#x["Embarked"] = le_embarked.transform(x["Embarked"])

In [285]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x,y,test_size=0.33,stratify=y)

In [286]:
# Normalización, test, recodificacion, eliminar variables innecesarias

## Árboles de decisión - Evaluación

In [287]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion="gini",min_samples_split=10,min_samples_leaf=10)
tree.fit(train_x,train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [288]:
predict_y = tree.predict(test_x)

In [289]:
np.sum(predict_y == test_y)/test_y.shape[0]

0.8338983050847457

## Árboles de decisión - Public Test

In [290]:
tree.fit(x,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [292]:
test = pd.read_csv("test.csv")
test.fillna(test.mean(),inplace=True)
test_x = test.drop(["Name","Ticket","PassengerId","Embarked"],axis=1)
test_x["Sex"] = le_sex.transform(test_x["Sex"])
#test_x["Embarked"] = test_x["Embarked"].apply(lambda x: "nan" if pd.isnull(x) else x)
#test_x["Embarked"] = le_embarked.transform(test_x["Embarked"])
test_x["Cabin"] = test_x["Cabin"].apply(lambda x: 0 if pd.isnull(x) else 1)

In [293]:
predict_y = tree.predict(test_x)

In [294]:
output = pd.DataFrame()
output["PassengerId"] = test["PassengerId"].iloc[range(0,test.shape[0])]
output["Survived"] = predict_y

In [295]:
output.to_csv("output.csv",index=False)